In [1]:
path = "C:\\users\\Galia\\desktop\\kaggle\\data\\egonets"

In [2]:
import networkx as nx

In [2]:
from os import listdir
from os.path import isfile, join

In [3]:
import itertools

In [4]:
import matplotlib.pyplot as plt
import os
import re
import scipy 
from scipy.sparse import *
from operator import itemgetter
from sklearn.cluster import KMeans
import numpy as np
import sys
import pandas as pd

In [116]:
#The function takes as argument an integer in the range 0-109 and path where egonets are located 
#returns a networkx graph of the friends network of the user
def build_graph(n, path):
    try:
        onlyfiles = [fyle for fyle in listdir(path) if fyle.endswith('.egonet')]
    except:
        print("Problems with detecring files UserID.egonets at the specified path")
    edges = []
    nodes = []
    egonets = onlyfiles
    egonet = egonets[n]
    ego = int(re.match( r'([0-9]+).egonet', egonet).group(1))
    m = open(os.path.join(path,egonet), "r")
    friends = [line[:-1].replace(":", "").split(" ") for line in m.readlines()]
    friends = list(list(map(int, friend[:1])) if friend[1] == '' else list(map(int, friend)) for friend in friends)
    edges += [(ego,friend[0]) for friend in friends]
    for friend in friends:
        edges += [(friend[0], user) for user in friend[1:] if len(friend)>1] 
    nodes += list(itertools.chain.from_iterable(friends)) + [ego]
    edges = list(set(tuple(sorted(edge)) for edge in edges))        
    nodes = list(set(nodes))
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)
    
    return G, nodes


In [117]:
#generates a CSV file with the Adjacency matrix representation of G

def to_R(n, path):
    G, nodes = build_graph(n, path)
    A = nx.to_numpy_matrix(G)
    df = pd.DataFrame(A, index = nodes, columns = nodes)
    df.to_csv('graph-{}.csv'.format(n),index = True)

In [120]:
if __name__ == '__main__':
    for n in range(110):
        to_R(n, path)